In [1]:
from aif360.metrics.classification_metric import ClassificationMetric
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings

from utils import make_dataset, log_fairness_metrics

warnings.filterwarnings('ignore')

TARGET_COLS = 'two_year_recid'
COMPAS_SCORES_COLS = 'decile_score'
NUMERICAL_FEATURE_COLS = ['age',
                          'juv_fel_count','juv_misd_count','juv_other_count',
                          'priors_count','jail_time']
CATEGORICAL_FEATURE_COLS = ['sex','race',
                            'c_charge_degree']
FEATURE_NAMES = NUMERICAL_FEATURE_COLS+CATEGORICAL_FEATURE_COLS

PROTECTED_COLS = ['sex','race']

BIAS_INFO = {'favorable_label':0,
             'unfavorable_label':1,
             'protected_columns':['race'],
            }

PRIVILEGED_INFO = {'unprivileged_groups':[{'race': 2},
                                          {'race': 1},
                                          {'race': 4},
                                          {'race': 5},
                                          {'race': 6}],
                   'privileged_groups':[{'race': 3}]
                  }

data = pd.read_csv('../data/processed/compas-scores-two-years-processed.csv')
data.head()

/home/kuba/programs/miniconda3/envs/model_fairness/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/kuba/programs/miniconda3/envs/model_fairness/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,jail_time,sex,race,c_charge_degree,decile_score,two_year_recid,compas_score
0,69,0,0,0,0,0.0,1,1,1,1,0,0.1
1,34,0,0,0,0,10.0,1,2,1,3,1,0.3
2,24,0,0,1,4,1.0,1,2,1,4,1,0.4
3,23,0,1,0,1,0.0,1,2,1,8,0,0.8
4,43,0,0,0,2,0.0,1,1,1,1,0,0.1


### Disparate Impact Remover

In [2]:
from aif360.algorithms.preprocessing import DisparateImpactRemover

dataset = make_dataset(data, 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)

disp_imp_remover = DisparateImpactRemover(sensitive_attribute='race',repair_level=1.0)

dataset = disp_imp_remover.fit_transform(dataset).convert_to_dataframe()[0]

In [3]:
data_cleaned = data.copy()
data_cleaned['race'] = dataset['race'].values

In [4]:
train, test = train_test_split(data_cleaned, test_size=0.2, random_state=1234)

X_train, y_train = train[FEATURE_NAMES], train[TARGET_COLS]
X_test, y_test = test[FEATURE_NAMES], test[TARGET_COLS]

In [5]:
clf = LogisticRegression(random_state=1234)
clf.fit(X_train, y_train)

y_test_pred = clf.predict_proba(X_test)
test['recid_prediction'] = (y_test_pred[:,1] >0.5).astype(int)

roc_auc = roc_auc_score(y_test, y_test_pred[:,1])

In [6]:
ground_truth_test = make_dataset(test, 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)
prediction_test = make_dataset(test, 'recid_prediction', **BIAS_INFO, **PRIVILEGED_INFO)

clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)

In [7]:
import neptune
from neptunecontrib.api.utils import get_filepaths

neptune.init('jakub-czakon/model-fairness')
neptune.create_experiment(name='baseline',tags=['preprocessing','race','disparate-impact-remover'], 
                          upload_source_files=get_filepaths(extensions=['.py','.ipynb']))

neptune.log_metric('roc_auc',roc_auc)
log_fairness_metrics(clf_metric)

neptune.stop()

https://ui.neptune.ml/jakub-czakon/model-fairness/e/MOD-84


# Reweighing

In [8]:
from aif360.algorithms.preprocessing import Reweighing

reweighing = Reweighing(**PRIVILEGED_INFO)

In [9]:
train, test = train_test_split(data, test_size=0.2, random_state=1234)

train_dataset = make_dataset(train, 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)
test_dataset = make_dataset(test, 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)

train_dataset = reweighing.fit_transform(train_dataset)
test_dataset = reweighing.transform(test_dataset)

In [10]:
train_dataset.instance_weights

array([1.06614556, 1.06614556, 0.9307455 , ..., 1.06614556, 0.89397015,
       0.9307455 ])

In [11]:
train['race']=train_dataset.convert_to_dataframe()[0]['race'].values
test['race']=test_dataset.convert_to_dataframe()[0]['race'].values

train_weights = train_dataset.instance_weights

In [12]:
X_train, y_train = train[FEATURE_NAMES], train[TARGET_COLS]
X_test, y_test = test[FEATURE_NAMES], test[TARGET_COLS]

In [13]:
clf = LogisticRegression(random_state=1234)
clf.fit(X_train, y_train, sample_weight=train_weights)

y_test_pred = clf.predict_proba(X_test)
test['recid_prediction'] = (y_test_pred[:,1] >0.5).astype(int)

roc_auc = roc_auc_score(y_test, y_test_pred[:,1])

In [14]:
ground_truth_test = make_dataset(test, 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)
prediction_test = make_dataset(test, 'recid_prediction', **BIAS_INFO, **PRIVILEGED_INFO)

clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)

In [16]:
import neptune
from neptunecontrib.api.utils import get_filepaths

neptune.init('jakub-czakon/model-fairness')
neptune.create_experiment(name='baseline',tags=['preprocessing','race','reweighing'], 
                          upload_source_files=get_filepaths(extensions=['.py','.ipynb']))

neptune.log_metric('roc_auc',roc_auc)
log_fairness_metrics(clf_metric)

neptune.stop()

https://ui.neptune.ml/jakub-czakon/model-fairness/e/MOD-86


# LFR

# OPTIM